In [ ]:
!pip install langchain chromadb python-dotenv sentence-transformers langchain-community langchain-huggingface

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from chromadb import Client
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from dotenv import load_dotenv
import os

# Update the TextLoader to specify the encoding
loader = TextLoader('dataset.txt', encoding='utf-8')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings()

# The rest of your code...


In [ ]:
embeddings

In [ ]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from chromadb import Client
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Define Index Name
index_name = "infinity-demo"

# Ensure the persist directory exists and has write permissions
persist_directory = "./chroma_data" 
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

# Check if the directory has write permissions
if not os.access(persist_directory, os.W_OK):
    raise PermissionError(f"The directory {persist_directory} is not writable.")

# Initialize Chroma client
chroma_client = Client(Settings(persist_directory=persist_directory))



# Check if the collection already exists
existing_collections = [collection.name for collection in chroma_client.list_collections()]

if index_name not in existing_collections:
    # Create new collection and add documents
    docsearch = Chroma.from_documents(documents=docs, embedding=embeddings, collection_name=index_name, persist_directory=persist_directory)
else:
    # Load the existing collection
    docsearch = Chroma(collection_name=index_name, persist_directory=persist_directory)

# Persist the index to disk
docsearch.persist()

print(f"Index '{index_name}' created and persisted successfully.")


In [22]:
# from langchain.llms import HuggingFaceHub

# Define the repo ID and connect to Mixtral model on Huggingface
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
llm = HuggingFaceHub(
  repo_id=repo_id, 
  model_kwargs={"temperature": 0.8, "top_k": 50, "top_p": 0.95, "max_length": 1000}, 
  huggingfacehub_api_token=api_token
)

In [23]:
from langchain import PromptTemplate

template = """
You are a fortune teller. These Human will ask you a questions about their life. 
Use following piece of context to answer the question. 
If you don't know the answer, just say you don't know. 
Keep the answer within 3 sentences and concise.
The answer can be up to 1000 words. If the enough 3 sentences then stop. 


Context: {context}
Question: {question}
Answer: 

"""

prompt = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

In [24]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
# import load env
from dotenv import load_dotenv
class ChatBot():
  load_dotenv()

  # The rest of the code here
  print(docsearch.as_retriever())

  rag_chain = (
    {"context": docsearch.as_retriever(),  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
  )

In [28]:
# Outside ChatBot() class
bot = ChatBot()
input = "nguyên nhân bệnh tử kỷ là gì" # convert to vector
result = bot.rag_chain.invoke(input)


In [ ]:
print(result)